***GENERATED CODE FOR fetalhealthclassification PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(df, path):
        df.to_csv("/dbfs" + eval(config)['url'], header=True, index=False)
        return True


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run fetalhealthclassificationHooks.ipynb
try:
	#sourcePreExecutionHook()

	fetalhealth = DBFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/1634802858065_fetal_health.csv', 'filename': '1634802858065_fetal_health.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/Healthcare Models/Fetal Health - Classification/fetal_health.csv', 'viewFileName': 'fetal_health.csv', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/', 'results_url': 'http://52.142.62.122:4040/api/read/hdfs'}")
	#sourcePostExecutionHook(fetalhealth)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run fetalhealthclassificationHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(fetalhealth,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "baseline value", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "133.69", "stddev": "10.16", "min": "116", "max": "158", "missing": "0"}, "updatedLabel": "baseline value"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "accelerations", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "42", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.014", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "accelerations"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "fetal_movement", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "42", "mean": "0.01", "stddev": "0.01", "min": "0.0", "max": "0.048", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "fetal_movement"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "uterine_contractions", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "42", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.013", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "uterine_contractions"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "light_decelerations", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "42", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.015", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "light_decelerations"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "severe_decelerations", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "severe_decelerations"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "prolongued_decelerations", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "42", "mean": "0.0", "stddev": "0.0", "min": "0.0", "max": "0.003", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "prolongued_decelerations"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "abnormal_short_term_variability", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "48.33", "stddev": "16.0", "min": "22", "max": "84", "missing": "0"}, "updatedLabel": "abnormal_short_term_varia..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "mean_value_of_short_term_variability", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "42", "mean": "1.31", "stddev": "1.32", "min": "0.2", "max": "6.3", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "mean_value_of_short_term_..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "percentage_of_time_with_abnormal_long_term_variability", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "12.6", "stddev": "19.8", "min": "0", "max": "81", "missing": "0"}, "updatedLabel": "percentage_of_time_with_a..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "mean_value_of_long_term_variability", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "42", "mean": "9.31", "stddev": "5.85", "min": "0.0", "max": "27.6", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "mean_value_of_long_term_v..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "histogram_width", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "67.5", "stddev": "43.05", "min": "10", "max": "150", "missing": "0"}, "updatedLabel": "histogram_width"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "histogram_min", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "97.5", "stddev": "33.8", "min": "50", "max": "153", "missing": "0"}, "updatedLabel": "histogram_min"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "histogram_max", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "165.0", "stddev": "20.33", "min": "131", "max": "200", "missing": "0"}, "updatedLabel": "histogram_max"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "histogram_number_of_peaks", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "3.83", "stddev": "2.89", "min": "0", "max": "11", "missing": "0"}, "updatedLabel": "histogram_number_of_peaks"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "histogram_number_of_zeroes", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "0.33", "stddev": "0.69", "min": "0", "max": "3", "missing": "0"}, "updatedLabel": "histogram_number_of_zeroe..."}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "histogram_mode", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "139.67", "stddev": "18.65", "min": "71", "max": "187", "missing": "0"}, "updatedLabel": "histogram_mode"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "histogram_mean", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "138.36", "stddev": "13.97", "min": "107", "max": "178", "missing": "0"}, "updatedLabel": "histogram_mean"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "histogram_median", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "140.07", "stddev": "13.88", "min": "106", "max": "180", "missing": "0"}, "updatedLabel": "histogram_median"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "histogram_variance", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "18.43", "stddev": "40.19", "min": "0", "max": "215", "missing": "0"}, "updatedLabel": "histogram_variance"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "histogram_tendency", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "0.29", "stddev": "0.6", "min": "-1", "max": "1", "missing": "0"}, "updatedLabel": "histogram_tendency"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "fetal_health", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "42", "mean": "1.29", "stddev": "0.55", "min": "1", "max": "3", "missing": "0"}, "updatedLabel": "fetal_health"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run fetalhealthclassificationHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(autofe, ["baseline value", "accelerations", "uterine_contractions", "light_decelerations", "severe_decelerations", "prolongued_decelerations", "abnormal_short_term_variability", "mean_value_of_short_term_variability", "percentage_of_time_with_abnormal_long_term_variability", "mean_value_of_long_term_variability", "histogram_width", "histogram_min", "histogram_max", "histogram_number_of_peaks", "histogram_number_of_zeroes", "histogram_mode", "histogram_mean", "histogram_median", "histogram_variance", "histogram_tendency", "fetal_health"], "fetal_movement")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

